In [1]:
from cyvcf2 import VCF, Writer
import pandas as pd
import numpy as np

In [3]:
### Get the Benchmark file
truth = pd.read_csv("/hpc/umc_laat/gvandersluis/data/Ont_data_nhung/HG002/wf_hvar_ROI/HG002_BM_SSANDT_rn.vcf.gz", compression="gzip", comment="#", sep = "\t", names=["CHROM", "POS", "ID", "REF", "ALT", "QUAL", "FILTER", "INFO", "FORMAT", "sample"]).drop("INFO", axis=1)

In [4]:
### Get and generate the necessary columns
truth["GT_BM"] = truth["sample"].str.split(":").str[0]
truth["tag"] = truth["CHROM"].astype(str)+"_"+truth["POS"].astype(str)
short_truth = truth[["tag", "GT_BM"]]
short_truth

,tag,GT_BM
0,chr1_602113,1/1
1,chr1_604358,1/1
2,chr1_604360,1/1
3,chr1_611317,1/1
4,chr1_631859,1/1
...,...,...
4048337,chr22_50791171,1|0
4048338,chr22_50791276,0/1
4048339,chr22_50791289,0/1
4048340,chr22_50792075,1|0


In [5]:
### Remove the variants that contain more variation possibilities '2' and / (unphased variants)
short_truth = short_truth[(short_truth["GT_BM"].str.contains("2") == False) & (short_truth["GT_BM"].str.contains("/") == False)]
short_truth

,tag,GT_BM
138,chr1_885657,1|0
161,chr1_908025,1|0
170,chr1_916010,0|1
204,chr1_928128,0|1
208,chr1_928253,0|1
...,...,...
4048329,chr22_50787086,1|0
4048331,chr22_50787916,1|0
4048333,chr22_50789463,1|0
4048337,chr22_50791171,1|0


In [7]:
vcf = pd.read_csv('/hpc/umc_laat/gvandersluis/data/Ont_data_nhung/HG002/wf_hvar_ROI/sub_PStag.csv')
vcf

,CHROM,POS,QUAL,GT,GQ,DP,PS,REF,ALT,ROI,PS_tag
0,chr1,434272,4.07,0|1,4,33,413640,A,C,chr1:434249-4371348,chr1_413640
1,chr1,434292,20.74,0|1,20,33,413640,T,C,chr1:434249-4371348,chr1_413640
2,chr1,434325,13.74,0|1,13,33,413640,C,G,chr1:434249-4371348,chr1_413640
3,chr1,434364,16.92,1|0,16,33,413640,A,G,chr1:434249-4371348,chr1_413640
4,chr1,434451,23.76,0|1,23,33,413640,G,A,chr1:434249-4371348,chr1_413640
...,...,...,...,...,...,...,...,...,...,...,...
2301224,chrY,26117052,2.78,1|0,2,7,26116800,G,T,chrY:24288280-26123509,chrY_26116800
2301225,chrY,26117825,5.52,1|0,5,6,26116800,C,T,chrY:24288280-26123509,chrY_26116800
2301226,chrY,26118416,4.62,1|0,4,6,26116800,T,A,chrY:24288280-26123509,chrY_26116800
2301227,chrY,26119317,3.20,1|0,3,5,26116800,TGAA,T,chrY:24288280-26123509,chrY_26116800


In [10]:
### Load the variant files, collect the necessary columns and remove unphased variants
vcf = pd.read_csv('/hpc/umc_laat/gvandersluis/data/Ont_data_nhung/HG002/wf_hvar_ROI/rn_wf_hvar.wf_snp.vcf.gz', comment="#", sep = "\t", names=["CHROM", "POS", "ID", "REF", "ALT", "QUAL", "FILTER", "INFO", "FORMAT", "sample"])
vcf["GT"] = vcf["sample"].str.split(":").str[0]
vcf = vcf[vcf["GT"].str.contains("/") == False]
vcf["Cor_phased"]="unknown"
vcf["PS_tag"] = vcf["CHROM"].astype(str)+"_"+vcf["sample"].str.split(":").str[5]
vcf["tag"] = vcf["CHROM"].astype(str)+"_"+vcf["POS"].astype(str)

vcf


,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,sample,GT,Cor_phased,PS_tag,tag
0,chr1,434272,.,A,C,4.07,PASS,F,GT:GQ:DP:AD:AF:PS,"0|1:4:33:18,13:0.3939:413640",0|1,unknown,chr1_413640,chr1_434272
2,chr1,434292,.,T,C,20.74,PASS,F,GT:GQ:DP:AD:AF:PS,"0|1:20:33:17,14:0.4242:413640",0|1,unknown,chr1_413640,chr1_434292
3,chr1,434325,.,C,G,13.74,PASS,F,GT:GQ:DP:AD:AF:PS,"0|1:13:33:8,21:0.6364:413640",0|1,unknown,chr1_413640,chr1_434325
4,chr1,434364,.,A,G,16.92,PASS,F,GT:GQ:DP:AD:AF:PS,"1|0:16:33:24,8:0.2424:413640",1|0,unknown,chr1_413640,chr1_434364
5,chr1,434451,.,G,A,23.76,PASS,F,GT:GQ:DP:AD:AF:PS,"0|1:23:33:18,15:0.4545:413640",0|1,unknown,chr1_413640,chr1_434451
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3912429,chrY,26117052,.,G,T,2.78,PASS,F,GT:GQ:DP:AD:AF:PS,"1|0:2:7:0,6:0.8571:26116800",1|0,unknown,chrY_26116800,chrY_26117052
3912430,chrY,26117825,.,C,T,5.52,PASS,F,GT:GQ:DP:AD:AF:PS,"1|0:5:6:1,5:0.8333:26116800",1|0,unknown,chrY_26116800,chrY_26117825
3912431,chrY,26118416,.,T,A,4.62,PASS,F,GT:GQ:DP:AD:AF:PS,"1|0:4:6:0,5:0.8333:26116800",1|0,unknown,chrY_26116800,chrY_26118416
3912432,chrY,26119317,.,TGAA,T,3.20,PASS,F,GT:GQ:DP:AD:AF:PS,"1|0:3:5:3,2:0.4:26116800",1|0,unknown,chrY_26116800,chrY_26119317


In [12]:
### Merge the Benchmark file with the VCF file on the generated location tag 'tag'
### Turn all variants of the vcf that are in the benchmark to True. 
### For haploblocks without switches, this will mean that all variants are correctly phased that are found in the benchmark file
complete = pd.merge(vcf, short_truth, on="tag", how="left")
complete.loc[complete["GT_BM"].notna(), "Cor_phased"] = "True"
complete

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,sample,GT,Cor_phased,PS_tag,tag,GT_BM
0,chr1,434272,.,A,C,4.07,PASS,F,GT:GQ:DP:AD:AF:PS,"0|1:4:33:18,13:0.3939:413640",0|1,unknown,chr1_413640,chr1_434272,NaN
1,chr1,434292,.,T,C,20.74,PASS,F,GT:GQ:DP:AD:AF:PS,"0|1:20:33:17,14:0.4242:413640",0|1,unknown,chr1_413640,chr1_434292,NaN
2,chr1,434325,.,C,G,13.74,PASS,F,GT:GQ:DP:AD:AF:PS,"0|1:13:33:8,21:0.6364:413640",0|1,unknown,chr1_413640,chr1_434325,NaN
3,chr1,434364,.,A,G,16.92,PASS,F,GT:GQ:DP:AD:AF:PS,"1|0:16:33:24,8:0.2424:413640",1|0,unknown,chr1_413640,chr1_434364,NaN
4,chr1,434451,.,G,A,23.76,PASS,F,GT:GQ:DP:AD:AF:PS,"0|1:23:33:18,15:0.4545:413640",0|1,unknown,chr1_413640,chr1_434451,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2301224,chrY,26117052,.,G,T,2.78,PASS,F,GT:GQ:DP:AD:AF:PS,"1|0:2:7:0,6:0.8571:26116800",1|0,unknown,chrY_26116800,chrY_26117052,NaN
2301225,chrY,26117825,.,C,T,5.52,PASS,F,GT:GQ:DP:AD:AF:PS,"1|0:5:6:1,5:0.8333:26116800",1|0,unknown,chrY_26116800,chrY_26117825,NaN
2301226,chrY,26118416,.,T,A,4.62,PASS,F,GT:GQ:DP:AD:AF:PS,"1|0:4:6:0,5:0.8333:26116800",1|0,unknown,chrY_26116800,chrY_26118416,NaN
2301227,chrY,26119317,.,TGAA,T,3.20,PASS,F,GT:GQ:DP:AD:AF:PS,"1|0:3:5:3,2:0.4:26116800",1|0,unknown,chrY_26116800,chrY_26119317,NaN


In [14]:
### From the haploblock file, identify the PS_tags that contain switches. 
### For these haploblocks the variants need to be determined if they are correctly phased
v5_2_new = pd.read_csv("/hpc/umc_laat/gvandersluis/data/Ont_data_nhung/HG002/wf_hvar_ROI/Haploblock_switches.csv")
ps_tags = v5_2_new[(v5_2_new["all_switches"] > 0) & (v5_2_new["all_switches"].notna())].sort_values("phased_variants")["PS_tag"].unique()

v5_2_new


,PS_tag,START_HB,END_HB,phased_variants,chromosome,GQ_mean,HB_length,total_VARS,het_variants0,all_switches,all_switch_rate,all_switchflips,all_switchflip_rate,blockwise_hamming_rate,Accuracy
0,chr10_100030138,100030138,100092179,6,chr10,30.833333,62041,110,2.0,0.0,0.0,0/0,0.0,0.0,100.0%
1,chr10_100140109,100127172,101751973,1484,chr10,47.721698,1624801,2445,1285.0,0.0,0.0,0/0,0.0,0.0,100.0%
2,chr10_101790524,101770141,102457835,310,chr10,47.070968,687694,559,279.0,0.0,0.0,0/0,0.0,0.0,100.0%
3,chr10_102503756,102503756,103822548,1246,chr10,50.865169,1318792,1946,1160.0,0.0,0.0,0/0,0.0,0.0,100.0%
4,chr10_103875639,103875639,104794626,592,chr10,49.162162,918987,1269,526.0,0.0,0.0,0/0,0.0,0.0,100.0%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3973,chrY_9055695,9055695,9057587,140,chrY,31.935714,1892,140,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3974,chrY_9107650,9107650,9109819,93,chrY,29.010753,2169,93,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3975,chrY_9336595,9336595,9403472,176,chrY,18.755682,66877,229,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3976,chrY_9454189,9454189,9537326,239,chrY,16.146444,83137,326,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
ps_tags

array(['chr4_104124010', 'chr21_45671324', 'chr1_158235312',
       'chr6_41166399', 'chr11_48108397', 'chr15_74812627',
       'chr10_73148535', 'chr4_99089583', 'chr12_86305043',
       'chr2_219155409', 'chr7_113958227.A', 'chr12_102076726',
       'chr1_248299047', 'chr8_33168496', 'chr7_134017344',
       'chr20_46493310.B', 'chr16_31645183', 'chr15_75155815',
       'chr1_2746360.B', 'chr5_129078485', 'chr15_81731947',
       'chr11_77832989', 'chr13_45019215', 'chr4_74489930',
       'chr4_56443348', 'chr10_118904905', 'chr8_88163149.B',
       'chr21_25540509', 'chr12_83741070', 'chr5_77055483.B',
       'chr3_177284140', 'chr15_73913439', 'chr17_64945801',
       'chr8_14450516.C', 'chr3_170195235', 'chr2_152548019.B',
       'chr1_52934953', 'chr12_40355907.C', 'chr2_111410354',
       'chr5_79367335', 'chr7_27767936.A', 'chr18_6925389.C',
       'chr11_64386887', 'chr1_47489942', 'chr1_120098698',
       'chr7_71485908.A', 'chr16_50094259', 'chr5_88415802.A',
       'chr12_6

In [19]:
ps_tags[5:20]
ps_df = v5_2_new[v5_2_new["PS_tag"].isin(ps_tags[5:20])][["PS_tag", "chromosome", "START_HB", "END_HB"]]

In [20]:
ps_df

,PS_tag,chromosome,START_HB,END_HB
138,chr10_73148535,chr10,73143832,73260982
384,chr12_102076726,chr12,102076726,102497932
523,chr12_86305043,chr12,86305043,86488426
873,chr15_74812627,chr15,74768584,74933419
875,chr15_75155815,chr15,75147903,75379620
952,chr16_31645183,chr16,31645183,31779282
1499,chr1_248299047,chr1,248299047,248518453
1507,chr1_2746360.B,chr1,4655077,4784883
1677,chr20_46493310.B,chr20,47506177,47661572
2013,chr2_219155409,chr2,219152789,219329415


In [29]:

count = 0
### Run through all phases tags (haploblocks) in which switches are present
for i in ps_tags[10:20]:
    count += 1
    print(count)
    PS = ps_df[(ps_df["PS_tag"] == i)]
    print(i)
    ps_chrm = PS["chromosome"].iloc[0]
    ps_strt = PS["START_HB"].iloc[0]
    ps_end = PS["END_HB"].iloc[0]
    ### Filter rows of the dataframe that contain the PS tag and are not NaN
    subset = complete[(complete["CHROM"] == ps_chrm) & (complete["POS"] >= ps_strt) & (complete["POS"] <= ps_end) & (complete["GT_BM"].notna())]
    ### For variant in this cluster if it is not empty:
    if not subset.empty:
        for index, item in subset.iterrows():
            ### To check if the variants are phased correctly according to the benchmark an on which allele they are,
            ### an X or O is connected to the variant. And - as a check to see if something else is up
            if item["GT"] == item["GT_BM"]:
                complete.loc[index, "Cor_phased"] = "X"
            elif item["GT"] == item["GT_BM"][::-1]:
                complete.loc[index, "Cor_phased"] = "O"
            else:
                complete.loc[index, "Cor_phased"] = "-"
    ### Per haploblock we can now calculate the Hemming rate.
    tot = complete[(complete["CHROM"] == ps_chrm) & (complete["POS"] >= ps_strt) & (complete["POS"] <= ps_end)].groupby("Cor_phased").agg("count")
    tot_x = tot.loc[["X"],"GT"]
    tot_o = tot.loc[["O"],"GT"]
    total_vars = v5_2_new[v5_2_new["PS_tag"] == i]["het_variants0"]
    off_hemm_rate = v5_2_new[v5_2_new["PS_tag"] == i]["blockwise_hamming_rate"]
    hemm_rate_x = tot_x.loc["X"]/(tot_x.loc["X"] + tot_o.loc["O"])
    hemm_rate_o = tot_o.loc["O"]/(tot_x.loc["X"] + tot_o.loc["O"])

    ### If the hemming rate is higher with the X, then O variants are the switches and the other way around. 
    ### If the rate is 0.50, it is not possible to know which are phased correctly
    if round(hemm_rate_x,2) == 0.50:
        complete.loc[(complete["CHROM"] == ps_chrm) & (complete["POS"] >= ps_strt) & (complete["POS"] <= ps_end) & (complete["GT_BM"].notna()), "Cor_phased"] = "UNKNOWN"
    elif round(hemm_rate_x,4) > round(hemm_rate_o,4):
        complete.loc[(complete["CHROM"] == ps_chrm) & (complete["POS"] >= ps_strt) & (complete["POS"] <= ps_end) & (complete["Cor_phased"] == "O" ), "Cor_phased"] = "False"
        complete.loc[(complete["CHROM"] == ps_chrm) & (complete["POS"] >= ps_strt) & (complete["POS"] <= ps_end) & (complete["Cor_phased"] == "X" ), "Cor_phased"] = "True"
    else:
        complete.loc[(complete["CHROM"] == ps_chrm) & (complete["POS"] >= ps_strt) & (complete["POS"] <= ps_end) & (complete["Cor_phased"] == "X" ), "Cor_phased"] = "False"
        complete.loc[(complete["CHROM"] == ps_chrm) & (complete["POS"] >= ps_strt) & (complete["POS"] <= ps_end) & (complete["Cor_phased"] == "O" ), "Cor_phased"] = "True"



1
chr7_113958227.A
2
chr12_102076726
3
chr1_248299047
4
chr8_33168496
5
chr7_134017344
6
chr20_46493310.B
7
chr16_31645183
8
chr15_75155815
9
chr1_2746360.B
10
chr5_129078485


In [32]:
complete[complete["Cor_phased"] == "False"]

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,sample,GT,Cor_phased,PS_tag,tag,GT_BM
4918,chr1,4710386,.,T,C,53.72,PASS,F,GT:GQ:DP:AD:AF:PS,"0|1:53:46:18,28:0.6087:2746360",0|1,False,chr1_2746360,chr1_4710386,1|0
187607,chr1,248462256,.,C,T,64.05,PASS,F,GT:GQ:DP:AD:AF:PS,"0|1:64:52:26,26:0.5:248299047",0|1,False,chr1_248299047,chr1_248462256,0|1
187608,chr1,248462685,.,CAT,C,63.22,PASS,F,GT:GQ:DP:AD:AF:PS,"1|0:63:53:25,27:0.5094:248299047",1|0,False,chr1_248299047,chr1_248462685,1|0
187609,chr1,248465696,.,T,G,25.42,PASS,P,GT:GQ:DP:AD:AF:PS,"0|1:25:57:31,25:0.4386:248299047",0|1,False,chr1_248299047,chr1_248465696,0|1
187610,chr1,248469393,.,C,T,57.11,PASS,F,GT:GQ:DP:AD:AF:PS,"1|0:57:56:27,28:0.5:248299047",1|0,False,chr1_248299047,chr1_248469393,1|0
334716,chr2,219193082,.,G,GA,54.91,PASS,F,GT:GQ:DP:AD:AF:PS,"1|0:54:45:21,23:0.5111:219155409",1|0,False,chr2_219155409,chr2_219193082,0|1
334717,chr2,219199061,.,C,G,48.20,PASS,F,GT:GQ:DP:AD:AF:PS,"1|0:48:50:29,20:0.4:219155409",1|0,False,chr2_219155409,chr2_219199061,0|1
334718,chr2,219202319,.,C,T,53.58,PASS,F,GT:GQ:DP:AD:AF:PS,"1|0:53:45:27,18:0.4:219155409",1|0,False,chr2_219155409,chr2_219202319,0|1
581530,chr4,99074792,.,TATAATG,T,50.56,PASS,F,GT:GQ:DP:AD:AF:PS,"1|0:50:46:23,22:0.4783:99089583",1|0,False,chr4_99089583,chr4_99074792,1|0
581531,chr4,99089583,.,C,T,55.00,PASS,F,GT:GQ:DP:AD:AF:PS,"0|1:55:45:28,17:0.3778:99089583",0|1,False,chr4_99089583,chr4_99089583,0|1


In [32]:
complete.to_csv("/hpc/umc_laat/gvandersluis/data/Ont_data_nhung/HG002/SUP_v5.2_ROI/cor_phased.csv", sep='\t', index=False)

In [12]:
comp_tab[comp_tab["Cor_phased"] == "-"]

,Unnamed: 0,chrom,pos,id,ref_x,alt_x,qual,filter,info,format,sample,Cor_phased,GT,PS_tag,tag,GT_BM,ref_y,alt_y
93463,93463,chr1,114647173,.,G,A,13.79,PASS,F,GT:GQ:DP:AD:AF:PS,"1|0:13:57:11,26:0.4561:114486666",-,1|0,chr1_114486666,chr1_114647173,1/1,G,A
172380,172380,chr1,234278517,.,G,A,5.66,PASS,F,GT:GQ:DP:AD:AF:PS,"0|1:5:63:0,38:0.6032:233144051",-,0|1,chr1_233144051,chr1_234278517,1/1,G,A
1693914,1693914,chr13,71553444,.,C,A,3.33,PASS,F,GT:GQ:DP:AD:AF:PS,"1|0:3:41:0,26:0.6341:67041383",-,1|0,chr13_67041383,chr13_71553444,1/1,C,A
1948137,1948137,chr17,2493718,.,C,CA,12.61,PASS,F,GT:GQ:DP:AD:AF:PS,"1|0:12:40:5,13:0.325:506523",-,1|0,chr17_506523,chr17_2493718,1/1,C,CA
2071674,2071674,chr18,70688336,.,C,T,4.52,PASS,F,GT:GQ:DP:AD:AF:PS,"1|0:4:36:8,20:0.5556:69194727",-,1|0,chr18_69194727,chr18_70688336,1/1,C,T


In [36]:
i = "chr16_80524674"
v5_2_new[v5_2_new["PS_tag"] == i]

,PS,START_HB,END_HB,phased_variants,chromosome,GQ_mean,GENES_in_HB,HB_length,total_VARS,PS_tag,het_variants0,all_switches,all_switch_rate,all_switchflips,all_switchflip_rate,blockwise_hamming_rate,Accuracy
3036,80524674,80524674,87365010,11576,chr16,52.91612,[nan],6840336,19260,chr16_80524674,10381.0,2.0,0.000271,0/1,0.000136,0.000136,0.9998644619137979%


In [37]:
complete[complete["PS_tag"] == i].groupby("Cor_phased").agg("count")

,chrom,pos,id,ref_x,alt_x,qual,filter,info,format,sample,GT,PS_tag,tag,GT_BM,ref_y,alt_y
Cor_phased,,,,,,,,,,,,,,,,
False,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
True,7377,7377,7377,7377,7377,7377,7377,7377,7377,7377,7377,7377,7377,7377,7377,7377
unknown,4198,4198,4198,4198,4198,4198,4198,4198,4198,4198,4198,4198,4198,0,0,0


In [38]:
complete[(complete["PS_tag"] == i)].drop("tag", axis=1)#["GT_BM"].unique()

,chrom,pos,id,ref_x,alt_x,qual,filter,info,format,sample,Cor_phased,GT,PS_tag,GT_BM,ref_y,alt_y
1929589,chr16,80524674,.,C,A,33.77,PASS,F,GT:GQ:DP:AD:AF:PS,"0|1:33:50:33,15:0.3:80524674",True,0|1,chr16_80524674,0|1,C,A
1929590,chr16,80525096,.,A,G,13.37,PASS,F,GT:GQ:DP:AD:AF:PS,"0|1:13:52:35,16:0.3077:80524674",True,0|1,chr16_80524674,0|1,A,G
1929591,chr16,80525762,.,T,A,46.81,PASS,F,GT:GQ:DP:AD:AF:PS,"0|1:46:49:33,16:0.3265:80524674",True,0|1,chr16_80524674,0|1,T,A
1929592,chr16,80525816,.,C,T,57.22,PASS,F,GT:GQ:DP:AD:AF:PS,"1|0:57:49:15,32:0.6531:80524674",True,1|0,chr16_80524674,1|0,C,T
1929593,chr16,80526026,.,C,G,47.61,PASS,F,GT:GQ:DP:AD:AF:PS,"1|0:47:49:15,33:0.6735:80524674",True,1|0,chr16_80524674,1|0,C,G
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1941160,chr16,87347460,.,T,C,62.08,PASS,F,GT:GQ:DP:AD:AF:PS,"0|1:62:59:28,30:0.5085:80524674",True,0|1,chr16_80524674,0|1,T,C
1941161,chr16,87348238,.,G,A,60.79,PASS,F,GT:GQ:DP:AD:AF:PS,"0|1:60:58:28,30:0.5172:80524674",True,0|1,chr16_80524674,0|1,G,A
1941162,chr16,87349381,.,C,T,61.83,PASS,F,GT:GQ:DP:AD:AF:PS,"0|1:61:63:31,32:0.5079:80524674",True,0|1,chr16_80524674,0|1,C,T
1941163,chr16,87350052,.,T,C,69.69,PASS,F,GT:GQ:DP:AD:AF:PS,"0|1:69:64:32,32:0.5:80524674",unknown,0|1,chr16_80524674,NaN,NaN,NaN


In [39]:
complete[(complete["PS_tag"] == i) & (complete["Cor_phased"] == "False")].drop("tag", axis=1).sort_values("pos")

,chrom,pos,id,ref_x,alt_x,qual,filter,info,format,sample,Cor_phased,GT,PS_tag,GT_BM,ref_y,alt_y
1934003,chr16,83518437,.,G,A,66.93,PASS,F,GT:GQ:DP:AD:AF:PS,"0|1:66:42:16,26:0.619:80524674",False,0|1,chr16_80524674,1|0,G,A


In [40]:
complete[(complete["PS_tag"] == i) & (complete["GT_BM"].notna())]


,chrom,pos,id,ref_x,alt_x,qual,filter,info,format,sample,Cor_phased,GT,PS_tag,tag,GT_BM,ref_y,alt_y
1929589,chr16,80524674,.,C,A,33.77,PASS,F,GT:GQ:DP:AD:AF:PS,"0|1:33:50:33,15:0.3:80524674",True,0|1,chr16_80524674,chr16_80524674,0|1,C,A
1929590,chr16,80525096,.,A,G,13.37,PASS,F,GT:GQ:DP:AD:AF:PS,"0|1:13:52:35,16:0.3077:80524674",True,0|1,chr16_80524674,chr16_80525096,0|1,A,G
1929591,chr16,80525762,.,T,A,46.81,PASS,F,GT:GQ:DP:AD:AF:PS,"0|1:46:49:33,16:0.3265:80524674",True,0|1,chr16_80524674,chr16_80525762,0|1,T,A
1929592,chr16,80525816,.,C,T,57.22,PASS,F,GT:GQ:DP:AD:AF:PS,"1|0:57:49:15,32:0.6531:80524674",True,1|0,chr16_80524674,chr16_80525816,1|0,C,T
1929593,chr16,80526026,.,C,G,47.61,PASS,F,GT:GQ:DP:AD:AF:PS,"1|0:47:49:15,33:0.6735:80524674",True,1|0,chr16_80524674,chr16_80526026,1|0,C,G
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1941157,chr16,87344608,.,T,G,63.80,PASS,F,GT:GQ:DP:AD:AF:PS,"0|1:63:59:28,28:0.4746:80524674",True,0|1,chr16_80524674,chr16_87344608,0|1,T,G
1941158,chr16,87344782,.,G,C,62.61,PASS,F,GT:GQ:DP:AD:AF:PS,"0|1:62:59:31,28:0.4746:80524674",True,0|1,chr16_80524674,chr16_87344782,0|1,G,C
1941160,chr16,87347460,.,T,C,62.08,PASS,F,GT:GQ:DP:AD:AF:PS,"0|1:62:59:28,30:0.5085:80524674",True,0|1,chr16_80524674,chr16_87347460,0|1,T,C
1941161,chr16,87348238,.,G,A,60.79,PASS,F,GT:GQ:DP:AD:AF:PS,"0|1:60:58:28,30:0.5172:80524674",True,0|1,chr16_80524674,chr16_87348238,0|1,G,A


In [41]:
complete[complete["PS_tag"] == i]

,chrom,pos,id,ref_x,alt_x,qual,filter,info,format,sample,Cor_phased,GT,PS_tag,tag,GT_BM,ref_y,alt_y
1929589,chr16,80524674,.,C,A,33.77,PASS,F,GT:GQ:DP:AD:AF:PS,"0|1:33:50:33,15:0.3:80524674",True,0|1,chr16_80524674,chr16_80524674,0|1,C,A
1929590,chr16,80525096,.,A,G,13.37,PASS,F,GT:GQ:DP:AD:AF:PS,"0|1:13:52:35,16:0.3077:80524674",True,0|1,chr16_80524674,chr16_80525096,0|1,A,G
1929591,chr16,80525762,.,T,A,46.81,PASS,F,GT:GQ:DP:AD:AF:PS,"0|1:46:49:33,16:0.3265:80524674",True,0|1,chr16_80524674,chr16_80525762,0|1,T,A
1929592,chr16,80525816,.,C,T,57.22,PASS,F,GT:GQ:DP:AD:AF:PS,"1|0:57:49:15,32:0.6531:80524674",True,1|0,chr16_80524674,chr16_80525816,1|0,C,T
1929593,chr16,80526026,.,C,G,47.61,PASS,F,GT:GQ:DP:AD:AF:PS,"1|0:47:49:15,33:0.6735:80524674",True,1|0,chr16_80524674,chr16_80526026,1|0,C,G
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1941160,chr16,87347460,.,T,C,62.08,PASS,F,GT:GQ:DP:AD:AF:PS,"0|1:62:59:28,30:0.5085:80524674",True,0|1,chr16_80524674,chr16_87347460,0|1,T,C
1941161,chr16,87348238,.,G,A,60.79,PASS,F,GT:GQ:DP:AD:AF:PS,"0|1:60:58:28,30:0.5172:80524674",True,0|1,chr16_80524674,chr16_87348238,0|1,G,A
1941162,chr16,87349381,.,C,T,61.83,PASS,F,GT:GQ:DP:AD:AF:PS,"0|1:61:63:31,32:0.5079:80524674",True,0|1,chr16_80524674,chr16_87349381,0|1,C,T
1941163,chr16,87350052,.,T,C,69.69,PASS,F,GT:GQ:DP:AD:AF:PS,"0|1:69:64:32,32:0.5:80524674",unknown,0|1,chr16_80524674,chr16_87350052,NaN,NaN,NaN


In [112]:
tel=0
for index, i in complete.iterrows():
    if i["GT_BM"].notna():
        if i["GT"] == i["GT_BM"]:
            vcf.loc[index, "Cor_phased"] = "X"
        elif i["GT"] == truth.loc[truth["ID"] == i["tag"],"GT"].iloc[0][::-1]:
            vcf.loc[index, "Cor_phased"] = "O"
        else:
            vcf.loc[index, "Cor_phased"] = "-"

        tel += 1
        print(tel)
vcf

AttributeError: 'float' object has no attribute 'notna'

In [42]:
vcf[vcf["Cor_phased"] == "O"]


,chrom,pos,id,ref,alt,qual,filter,info,format,sample,Cor_phased,GT,PS,tag
127,chr1,885657,.,C,T,68.10,PASS,F,GT:GQ:DP:AD:AF:PS,"0|1:68:38:17,21:0.5526:789481",O,0|1,789481,chr1_885657
129,chr1,908025,.,A,G,50.80,PASS,F,GT:GQ:DP:AD:AF:PS,"0|1:50:40:20,18:0.45:789481",O,0|1,789481,chr1_908025
130,chr1,916010,.,G,T,61.18,PASS,F,GT:GQ:DP:AD:AF:PS,"1|0:61:38:18,18:0.4737:789481",O,1|0,789481,chr1_916010
131,chr1,928128,.,G,T,47.14,PASS,F,GT:GQ:DP:AD:AF:PS,"1|0:47:43:29,14:0.3256:789481",O,1|0,789481,chr1_928128
132,chr1,928253,.,G,A,50.02,PASS,F,GT:GQ:DP:AD:AF:PS,"1|0:50:43:29,14:0.3256:789481",O,1|0,789481,chr1_928253
133,chr1,933038,.,C,T,63.44,PASS,F,GT:GQ:DP:AD:AF:PS,"0|1:63:49:19,30:0.6122:789481",O,0|1,789481,chr1_933038
138,chr1,940387,.,T,G,49.05,PASS,F,GT:GQ:DP:AD:AF:PS,"1|0:49:56:33,23:0.4107:789481",O,1|0,789481,chr1_940387
139,chr1,940820,.,G,C,67.20,PASS,F,GT:GQ:DP:AD:AF:PS,"1|0:67:55:32,23:0.4182:789481",O,1|0,789481,chr1_940820
140,chr1,942934,.,G,C,65.09,PASS,F,GT:GQ:DP:AD:AF:PS,"0|1:65:50:21,26:0.52:789481",O,0|1,789481,chr1_942934
141,chr1,946870,.,C,A,51.09,PASS,F,GT:GQ:DP:AD:AF:PS,"0|1:51:44:22,22:0.5:789481",O,0|1,789481,chr1_946870


In [4]:
comp_tab = pd.read_csv("/hpc/umc_laat/gvandersluis/data/Ont_data_nhung/HG002/SUP_v5.2_ROI/cor_phased.csv", sep='\t')

In [7]:
ps = 0

comp_tab = pd.read_csv("/hpc/umc_laat/gvandersluis/data/Ont_data_nhung/HG002/SUP_v5.2_ROI/cor_phased.csv", sep='\t')

cor_ph = "/hpc/umc_laat/gvandersluis/data/Ont_data_nhung/HG002/SUP_v5.2_ROI/cor_phased.vcf.gz"
vcf_f = VCF("/hpc/umc_laat/gvandersluis/data/Ont_data_nhung/HG002/SUP_v5.2_ROI/roi_SUP_v5.2.wf_snp.vcf.gz")
vcf_f.add_format_to_header({
    'ID': 'Cor_phasing',
    'Description': 'indicates if the variant is switch error causing',
    'Type': 'String',
    'Number': '1'
})

w = Writer(comp_tab, vcf_f)
for var in comp_tab:
    w.write_record(var)
w.close()
vcf_f.close()

# num_samples = len(vcf_f.samples)
# cor_ph = []
# for record in vcf_f:
#     print(record.samples)
#     record.set_format("Cor_phasing", np.array(["UNKNWN"] * num_samples, dtype="S"))
#     if truth[truth['ID'] == f"{record.CHROM}_{record.POS}"].empty == False:
#         print(record.FORMAT['GT'].astype(str))
#         if record.FORMAT['GT'].astype(str) == truth.loc[truth["ID"] == f"{record.CHROM}_{record.POS}","GT"].iloc[0]:
#             cor_ph.append("X")
#         elif record.FORMAT['GT'].astype(str) == truth.loc[truth["ID"] == f"{record.CHROM}_{record.POS}","GT"].iloc[0][::-1]:
#             cor_ph.append("O")
#         else:
#             cor_ph.append("-")
#         print(record)
#     #print(f"{record.CHROM}:{record.POS} {record.REF}>{record.ALT}")


TypeError: Argument 'var' has incorrect type (expected cyvcf2.cyvcf2.Variant, got str)

In [ ]:
np.array(["UNKNWN"] * num_samples, dtype="S")

In [13]:
comp_tab.drop(("ref_y", "alt_y"), axis=1)

,Unnamed: 0,chrom,pos,id,ref_x,alt_x,qual,filter,info,format,sample,Cor_phased,GT,PS_tag,tag,GT_BM,ref_y,alt_y
0,0,chr1,434272,.,A,C,10.63,PASS,F,GT:GQ:DP:AD:AF:PS,"0|1:10:31:19,11:0.3548:413640",unknown,0|1,chr1_413640,chr1_434272,NaN,NaN,NaN
1,1,chr1,434287,.,G,A,13.04,PASS,F,GT:GQ:DP:AD:AF:PS,"1|0:13:31:17,12:0.3871:413640",unknown,1|0,chr1_413640,chr1_434287,NaN,NaN,NaN
2,2,chr1,434292,.,T,C,15.32,PASS,F,GT:GQ:DP:AD:AF:PS,"0|1:15:31:17,13:0.4194:413640",unknown,0|1,chr1_413640,chr1_434292,NaN,NaN,NaN
3,3,chr1,434451,.,G,A,17.59,PASS,F,GT:GQ:DP:AD:AF:PS,"0|1:17:31:18,13:0.4194:413640",unknown,0|1,chr1_413640,chr1_434451,NaN,NaN,NaN
4,4,chr1,434504,.,G,A,15.38,PASS,F,GT:GQ:DP:AD:AF:PS,"0|1:15:31:19,12:0.3871:413640",unknown,0|1,chr1_413640,chr1_434504,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2292089,2292089,chrY,25064188,.,G,A,9.68,PASS,F,GT:GQ:DP:AD:AF:PS,"0|1:9:23:16,6:0.2609:25064188",unknown,0|1,chrY_25064188,chrY_25064188,NaN,NaN,NaN
2292090,2292090,chrY,25065902,.,G,T,5.01,PASS,F,GT:GQ:DP:AD:AF:PS,"0|1:5:29:16,13:0.4483:25064188",unknown,0|1,chrY_25064188,chrY_25065902,NaN,NaN,NaN
2292091,2292091,chrY,26088251,.,TTATA,T,19.39,PASS,F,GT:GQ:DP:AD:AF:PS,"1|0:19:21:0,18:0.8571:26116800",unknown,1|0,chrY_26116800,chrY_26088251,NaN,NaN,NaN
2292092,2292092,chrY,26116800,.,C,G,9.34,PASS,F,GT:GQ:DP:AD:AF:PS,"1|0:9:7:2,4:0.5714:26116800",unknown,1|0,chrY_26116800,chrY_26116800,NaN,NaN,NaN


In [33]:
comp_tab = pd.read_csv(
    "/hpc/umc_laat/gvandersluis/data/Ont_data_nhung/HG002/SUP_v5.2_ROI/cor_phased.csv",
    sep="\t"
)

In [36]:
comp_tab = comp_tab[comp_tab["GT_BM"].notna()]

In [41]:

# # Load modified table
# comp_tab = pd.read_csv(
#     "/hpc/umc_laat/gvandersluis/data/Ont_data_nhung/HG002/SUP_v5.2_ROI/cor_phased.csv",
#     sep="\t"
# )
comp_tab = comp_tab.copy()

# Index for fast lookup
comp_tab["key"] = list(
    zip(comp_tab.CHROM, comp_tab.POS, comp_tab.REF, comp_tab.ALT)
)
lookup = dict(zip(comp_tab.key, comp_tab.Cor_phased))

# Open original VCF
vcf_f = VCF(
    "/hpc/umc_laat/gvandersluis/data/Ont_data_nhung/HG002/SUP_v5.2_ROI/roi_SUP_v5.2.wf_snp.vcf.gz"
)

vcf_f.add_format_to_header({
    "ID": "Cor_phased",
    "Description": "indicates if the variant is switch error causing",
    "Type": "String",
    "Number": "1"
})

writer = Writer(
    "/hpc/umc_laat/gvandersluis/data/Ont_data_nhung/HG002/SUP_v5.2_ROI/cor_phased.vcf.gz",
    vcf_f
)

# Transfer DataFrame data → VCF
for var in vcf_f:
    key = (var.CHROM, var.POS, var.REF, var.ALT[0])

    value = lookup.get(key, "NA")

    var.set_format(
        "Cor_phased",
        np.array([value.encode("ascii")], dtype="S")
    )

    writer.write_record(var)

writer.close()
vcf_f.close()


In [ ]:

def save_vcf(vcff, outf, comp_tab):
    comp_tab = comp_tab.copy()

    ### Index for fast lookup
    comp_tab["key"] = list(
        zip(comp_tab.CHROM, comp_tab.POS, comp_tab.REF, comp_tab.ALT)
    )
    lookup = dict(zip(comp_tab.key, comp_tab.Cor_phased))

    ### Open original VCF
    vcf_f = VCF(vcff)

    ### Edit the header to include Cor_phased
    vcf_f.add_format_to_header({
        "ID": "Cor_phased",
        "Description": "indicates if the variant is switch error causing",
        "Type": "String",
        "Number": "1"
    })

    ### Write vcf to output file
    writer = Writer(outf, vcf_f)

    ### For every variant, check dataframe and add Cor_phased Column
    for var in vcf_f:
        key = (var.CHROM, var.POS, var.REF, var.ALT[0])
        value = lookup.get(key, "NA")
        var.set_format("Cor_phased", np.array([value.encode("ascii")], dtype="S"))
        writer.write_record(var)
    writer.close()
    vcf_f.close()

def main():
    bm_file = "/hpc/umc_laat/gvandersluis/data/Ont_data_nhung/HG002/SUP_v5.2_ROI/HG002_BM_SSANDT_rn.vcf.gz"
    vcf_file = "/hpc/umc_laat/gvandersluis/data/Ont_data_nhung/HG002/SUP_v5.2_ROI/roi_SUP_v5.2.wf_snp.vcf.gz"
    haploblock_file = "/hpc/umc_laat/gvandersluis/data/Ont_data_nhung/HG002/SUP_v5.2_ROI/Haploblock_switches.csv"
    output_file = "/hpc/umc_laat/gvandersluis/data/Ont_data_nhung/HG002/SUP_v5.2_ROI/cor_phased.vcf.gz"
    bench_t = mod_bm(bm_file)
    comp_vcf = mod_vcf(vcf_file, bench_t)
    inf_tags = informative_tags(haploblock_file)
    corr_phased = corr_phased_vars(comp_vcf, inf_tags)
    save_vcf(vcf_file, output_file, corr_phased)


()